<a id='content'></a>

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Ознакомление-с-данными" data-toc-modified-id="Ознакомление-с-данными-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Ознакомление с данными</a></span></li><li><span><a href="#Запросы" data-toc-modified-id="Запросы-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Запросы</a></span><ul class="toc-item"><li><span><a href="#Посчитайте,-сколько-книг-вышло-после-1-января-2000-года" data-toc-modified-id="Посчитайте,-сколько-книг-вышло-после-1-января-2000-года-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Посчитайте, сколько книг вышло после 1 января 2000 года</a></span></li><li><span><a href="#Для-каждой-книги-посчитайте-количество-обзоров-и-среднюю-оценку" data-toc-modified-id="Для-каждой-книги-посчитайте-количество-обзоров-и-среднюю-оценку-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Для каждой книги посчитайте количество обзоров и среднюю оценку</a></span></li><li><span><a href="#Определите-издательство,-которое-выпустило-наибольшее-число-книг-толще-50-страниц-—-так-вы-исключите-из-анализа-брошюры" data-toc-modified-id="Определите-издательство,-которое-выпустило-наибольшее-число-книг-толще-50-страниц-—-так-вы-исключите-из-анализа-брошюры-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры</a></span></li><li><span><a href="#Определите-автора-с-самой-высокой-средней-оценкой-книг-—-учитывайте-только-книги-с-50-и-более-оценками" data-toc-modified-id="Определите-автора-с-самой-высокой-средней-оценкой-книг-—-учитывайте-только-книги-с-50-и-более-оценками-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками</a></span></li><li><span><a href="#Посчитайте-среднее-количество-обзоров-от-пользователей,-которые-поставили-больше-48-оценок" data-toc-modified-id="Посчитайте-среднее-количество-обзоров-от-пользователей,-которые-поставили-больше-48-оценок-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок</a></span></li></ul></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

# Подготовка

Загрузим необходимые библиотеки и подключимся к базе данных.

In [1]:
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

con=engine.connect()

# Сервис по чтению книг
[К содержанию](#content)

Анализируем данные крупного сервиса для чтения книг по подписке. **Цель исследования** - *получить и проанализировать данные, которые помогут сформулировать ценностное предложение для нового продукта*.

Для достижения цели нам следует выполнить следующую **задачу** - проанализировать базу данных, в которой хранится информация о книгах, издательствах, авторах, а также пользовательские обзоры книг.

## Ознакомление с данными
[К содержанию](#content)

In [3]:
books = '''
SELECT *
FROM books
LIMIT 1;
'''
pd.io.sql.read_sql(sql=text(books), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


Таблица `books` cодержит данные о книгах:
* book_id — идентификатор книги;
* author_id — идентификатор автора;
* title — название книги;
* num_pages — количество страниц;
* publication_date — дата публикации книги;
* publisher_id — идентификатор издателя.

In [4]:
authors = '''
SELECT *
FROM authors
LIMIT 1;
'''
pd.io.sql.read_sql(sql=text(authors), con = con)

,author_id,author
0,1,A.S. Byatt


Таблица `authors` - cодержит данные об авторах:
* author_id — идентификатор автора;
* author — имя автора.

In [5]:
ratings = '''
SELECT *
FROM ratings
LIMIT 1;
'''
pd.io.sql.read_sql(sql=text(ratings), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4


Таблица `ratings` - cодержит данные о пользовательских оценках книг:
* rating_id — идентификатор оценки;
* book_id — идентификатор книги;
* username — имя пользователя, оставившего оценку;
* rating — оценка книги.

In [6]:
reviews = '''
SELECT *
FROM reviews
LIMIT 1;
'''
pd.io.sql.read_sql(sql=text(reviews), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...


Таблица `reviews` - cодержит данные о пользовательских обзорах:
* review_id — идентификатор обзора;
* book_id — идентификатор книги;
* username — имя автора обзора;
* text — текст обзора.

In [7]:
publishers = '''
SELECT *
FROM publishers
LIMIT 1;
'''
pd.io.sql.read_sql(sql=text(publishers), con = con)

,publisher_id,publisher
0,1,Ace


Таблица `publishers` - cодержит данные об издательствах:
* publisher_id — идентификатор издательства;
* publisher — название издательства;

## Запросы
[К содержанию](#content)

Теперь напишем интересующие нас запросы

### Посчитайте, сколько книг вышло после 1 января 2000 года
[К содержанию](#content)

In [8]:
books_count = '''
SELECT COUNT(book_id)
FROM books
WHERE publication_date >='02-01-2000'; 
'''
pd.io.sql.read_sql(sql=text(books_count), con = con)

,count
0,819


После 1 января 2000 года вышло 819 книг.

### Для каждой книги посчитайте количество обзоров и среднюю оценку
[К содержанию](#content)

In [9]:
review_rating = '''
SELECT b.title AS title,
       b.book_id AS id,
       COUNT(DISTINCT re.review_id) AS review_cnt,
       AVG(ra.rating) AS rating
FROM books AS b
LEFT JOIN reviews AS re ON b.book_id = re.book_id
JOIN ratings AS ra ON b.book_id = ra.book_id
GROUP BY title, id
ORDER BY review_cnt DESC, rating DESC;
'''
pd.io.sql.read_sql(sql=text(review_rating), con=con)

,title,id,review_cnt,rating
0,Twilight (Twilight #1),948,7,3.662500
1,Harry Potter and the Prisoner of Azkaban (Harr...,302,6,4.414634
2,Harry Potter and the Chamber of Secrets (Harry...,299,6,4.287500
3,The Book Thief,656,6,4.264151
4,The Glass Castle,734,6,4.206897
...,...,...,...,...
995,Disney's Beauty and the Beast (A Little Golden...,191,0,4.000000
996,Leonardo's Notebooks,387,0,4.000000
997,Essential Tales and Poems,221,0,4.000000
998,Anne Rice's The Vampire Lestat: A Graphic Novel,83,0,3.666667


На первый взгляд оценки мало зависят от количества обзоров. Есть книги с высокими оценками, но c малым числом обзоров или вообще без обзоров. Больше всего обзоров написали на первую книгу Сумерек, однако оценка этой книги ниже 4.

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры
[К содержанию](#content)

In [10]:
publishers = '''
SELECT p.publisher AS publisher,
       COUNT(b.book_id) AS book_cnt
FROM publishers AS p
JOIN books AS b ON p.publisher_id = b.publisher_id
WHERE b.num_pages > 50
GROUP BY publisher
ORDER BY book_cnt DESC
LIMIT 1;
'''
pd.io.sql.read_sql(sql=text(publishers), con=con)

,publisher,book_cnt
0,Penguin Books,42


Активнее всего книги выпускало издательство Penguin Books - очень известное и крупное Британское издательство.

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками
[К содержанию](#content)

In [11]:
top_author = '''
WITH book_rating AS (SELECT a.author AS author_name,
                            b.book_id AS book,
                            AVG(ra.rating) AS avg_rate,
                            COUNT(ra.rating_id) AS rating_cnt
                     FROM authors AS a
                     JOIN books as b ON a.author_id=b.author_id
                     JOIN ratings AS ra ON b.book_id = ra.book_id
                     GROUP BY author_name, book
                     HAVING COUNT(ra.rating_id) >=50)

SELECT author_name,
       AVG(avg_rate) AS avg_rating
FROM book_rating
GROUP BY author_name
ORDER BY avg_rating DESC
LIMIT 1;
'''
pd.io.sql.read_sql(sql=text(top_author), con=con)

,author_name,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844


Самый высоко оцениваемый автор - J.K.Rowling/Mary GrandPre, средний рейтинг её книг составляет 4.3 балла. Что не удивительно, ведь она написала самую популярную и успешную франшизу книг.

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок
[К содержанию](#content)

In [12]:
users_review = '''
WITH rating_users AS (SELECT ra.username AS username,
                             COUNT(ra.rating_id) AS rate_cnt
                      FROM ratings AS ra
                      GROUP BY ra.username
                      HAVING COUNT(ra.rating_id) > 48),
     active_users AS (SELECT re.username,
                             COUNT(re.review_id) AS review_cnt
                      FROM reviews AS re
                      JOIN rating_users ON re.username = rating_users.username
                      GROUP BY re.username)

SELECT ROUND(AVG(review_cnt))
FROM active_users;
'''
pd.io.sql.read_sql(sql=text(users_review), con=con)

,round
0,24.0


В среднем, пользователи, которые оставляют более 48 оценок пишут 24 обзора.

## Вывод
[К содержанию](#content)

За период после 1 января 2000 года по настоящее (исследуемое) время на нашем сервисе вышло 819 книг. Самое активное издательство - Penguin Books. Самый высоко оцениваемый автор - J.K.Rowling.

На основании полученных данных можно составить следующее ценностное предложение касательно нашего приложения по чтению книг:
* Большая база доступных книг,
* Самых популярных и известных издательств,
* Представлены всеми любимые авторы,
* Перед чтением книги всегда можно ознакомиться с оценками и обзорами других пользователей.

Также, чтобы быть не отставать от конкурентов, можно добавить возможность прослушивания любимый книг в аудиоформате. А также предоставлять офлайн доступ к книгам.